# loading modules

In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sklearn.metrics import silhouette_score

from dca.api import dca
import anndata

from benchmarking import *
from helper import *

/home/ubuntu/dca/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/dca/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/dca/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/dca/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/dca/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Generating grammar tables from /usr/lib/python3.5/lib2to3/Grammar.txt
Generating grammar tables from /usr/lib/python3.5/lib2to3/PatternGrammar.txt


/home/ubuntu/dca/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/dca/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


# import data

In [2]:
# expression data
data_path = "/home/ubuntu/single-cell-scVI/data/10xPBMCs/"
expression_train = np.load(data_path + "de/data_train.npy")
expression_test = np.load(data_path + "de/data_test.npy")

# qc metrics
r_train = np.load(data_path + "design_train.npy")
r_test = np.load(data_path + "design_test.npy")

# labels
c_train = np.loadtxt(data_path + "label_train")
c_test = np.loadtxt(data_path + "label_test")

# batch info
b_train = np.loadtxt(data_path + "b_train")
b_test = np.loadtxt(data_path + "b_test")

# corrupted data
X_zero, i, j, ix = \
        np.load(data_path + "imputation/X_zero.npy"), np.load(data_path + "imputation/i.npy"),\
        np.load(data_path + "imputation/j.npy"), np.load(data_path + "imputation/ix.npy")

# Computational graph

In [5]:
train = anndata.AnnData(expression_train)
res = dca(train, verbose=True, mode="latent", hidden_size=(64, 10, 64))

DCA: Successfully preprocessed 3346 genes and 9029 cells.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
count (InputLayer)              (None, 3346)         0                                            
__________________________________________________________________________________________________
enc0 (Dense)                    (None, 64)           214208      count[0][0]                      
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 64)           192         enc0[0][0]                       
__________________________________________________________________________________________________
enc0_act (Activation)           (None, 64)           0           batch_normalization_4[0][0]      
___________________________________________________

In [6]:
# LATENT 10
latent = train.obsm['X_dca']
cluster_scores(latent, len(np.unique(c_train)), c_train)

[0.374523, 0.6921303361231197, 0.5549110822129342]

In [4]:
# LATENT 32 
latent = train.obsm['X_dca']
cluster_scores(latent, len(np.unique(c_train)), c_train)

[0.37623665, 0.7574129556811758, 0.6677622883585378]

### Imputation

In [9]:
train = anndata.AnnData(X_zero)
res = dca(train, verbose=True, mode="denoise")

DCA: Successfully preprocessed 3346 genes and 9029 cells.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
count (InputLayer)              (None, 3346)         0                                            
__________________________________________________________________________________________________
enc0 (Dense)                    (None, 64)           214208      count[0][0]                      
__________________________________________________________________________________________________
batch_normalization_7 (BatchNor (None, 64)           192         enc0[0][0]                       
__________________________________________________________________________________________________
enc0_act (Activation)           (None, 64)           0           batch_normalization_7[0][0]      
___________________________________________________

In [10]:
all_index = i[ix], j[ix]
x, y = train.X[all_index], expression_train[all_index]
np.median(np.abs(x - y))

0.8247501850128174

### denoise for DE

In [4]:
train = anndata.AnnData(expression_train)
res = dca(train, verbose=True, mode="denoise")

DCA: Successfully preprocessed 3346 genes and 9029 cells.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
count (InputLayer)              (None, 3346)         0                                            
__________________________________________________________________________________________________
enc0 (Dense)                    (None, 64)           214208      count[0][0]                      
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 64)           192         enc0[0][0]                       
__________________________________________________________________________________________________
enc0_act (Activation)           (None, 64)           0           batch_normalization_4[0][0]      
___________________________________________________

In [5]:
np.save(data_path + "de/dca_data_train", train.X.astype(np.int))